In [ ]:
%matplotlib inline
from datetime import datetime

import astropy.constants as const
import numpy as np

from sunpy.coordinates import get_horizons_coord
from sunpy.net import Fido, attrs as a
from sunpy.timeseries import TimeSeries

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from radiospectra import net
from radiospectra.spectrogram2 import Spectrogram
from stixpy.net.client import STIXClient
from stixpy.science import ScienceData

# Combined search for GOES XRS, EOVSA, and STIX data

In [ ]:
query = Fido.search(a.Time('2021-05-07T16:30', '2021-05-07T20:30'),
                    a.Instrument.xrs | a.Instrument.eovsa | 
                    (a.Instrument.stix & a.stix.DataProduct.sci_xray_spec))

In [ ]:
query

## Download specific results
* GOES 16, EOVSA Cross Pol ..

In [ ]:
goes_file = Fido.fetch(query[0][1])
eovsa_file = Fido.fetch(query[1][1])
stix_file = Fido.fetch(query[2][1])

## Load data in containers
* radiodospectra `Spectrogram` for EOVSA data
* sunpy `TimeSeries` for GOES XRS
* stixpy `SciencdData` for STIS

In [ ]:
eovsa_spec = Spectrogram(eovsa_file[0])
goes_timeseries = TimeSeries(goes_file[0])
stix_spec = ScienceData.from_fits(stix_file[0])

##  Light travel time

In [ ]:
solo_pos = get_horizons_coord('SOLO', time='2021-05-07T19:00')
earth_pos = get_horizons_coord('399', time='2021-05-07T19:00')
light_tt = (solo_pos.radius - earth_pos.radius)/const.c
light_tt.to('s')

## Modify STIX data
* Correct for light travel time to Earth
* Not recommended - will be included in software soon

In [ ]:
stix_spec.data['time'] = stix_spec.data['time'] - light_tt.to('s')

## Extact summed time series
* STIX 22-80 KeV
* EOVSA 12 - 15 GHz

In [ ]:
stix_data_22_70keV = stix_spec.get_data(energy_indices=[[16,26]], time_indices=(np.arange(612)*5).reshape(-1,2))
eovsa_fmask = np.argwhere((eovsa_spec.frequencies.value > 12) & (eovsa_spec.frequencies.value < 15))
eovsa_flux_12_15ghz = eovsa_spec.data[eovsa_fmask].sum(axis=0)

In [ ]:
stx_min, stix_max = stix_data_22_70keV[0].reshape(-1).value.min(), stix_data_22_70keV[0].reshape(-1).value.max()

In [ ]:
# Set up plot
fig, axes = plt.subplots(4, 1, sharex=True, dpi=100, figsize=(6, 9))

# Twin axes for STIX/GOES
ax0_twin = axes[0].twinx()

# Plot goes and STIX time series
goes_timeseries.plot(axes=axes[0])
stix_spec.plot_timeseries(energy_indices=[[1,10],[10,25]], axes=ax0_twin)

# Hack to add legends
goes_lines = axes[0].get_lines()
stix_lines = ax0_twin.get_lines()
goes_lines[0].set_color('k')
goes_lines[1].set_color('m')
axes[0].get_legend().remove()
axes[0].legend()
ax0_twin.legend(stix_lines, labels=['STIX 4-12 KeV', 'STIX 12-63 keV'])

# Plot STIX spectrogram
stix_spec.plot_spectrogram(axes=axes[1])
axes[1].set_yscale('log')

# Plot STIX spectrogram
eovsa_spec.plot(axes=axes[2], norm=LogNorm())
axes[2].set_title('')
axes[2].set_yscale('log')
axes[2].set_xlim(datetime(2021, 5, 7, 18, 48), datetime(2021, 5, 7, 19, 20))


# Plot EOVSA, STIX timeseris
axes[3].plot(eovsa_spec.times.datetime, eovsa_flux_12_15ghz.reshape(-1)/eovsa_flux_12_15ghz.reshape(-1).max(),
            label='EOVSA 12-15 GHz')
axes[3].plot((stix_data_22_70keV[2]).datetime, (stix_data_22_70keV[0].reshape(-1).value-stx_min)/(stix_max-stx_min),
             label='STIX 22-70 keV')
axes[3].legend()
axes[3].set_ylim(0, 1.2)
fig.subplots_adjust(hspace=0)